In [ ]:
import PythonicDISORT
import scipy as sc
import plotly as pl
import autograd as ag
import autograd.numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from numpy.polynomial.legendre import Legendre
from PythonicDISORT.subroutines import atleast_2d_append
from math import pi

# Matplotlib default colors
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

# Table of Contents
* [Load dataset](#Load-dataset)
* [Parameters and setup](#Parameters-and-setup)
	* [Figure 2](#Figure-2)
* [Exact coupling coefficients](#Exact-coupling-coefficients)
	* [Figure 1](#Figure-1)
* [Baseline two-stream closures](#Baseline-two-stream-closures)
	* [Figure 4](#Figure-4)
	* [Figure 9](#Figure-9)
* [Numerical Optimization (for one set of problem parameters)](#Numerical-Optimization-%28for-one-set-of-problem-parameters%29)
	* [Figure 5](#Figure-5)
* [Contour and 3D plots of dataset](#Contour-and-3D-plots-of-dataset)
	* [Figure 6](#Figure-6)
		* [Figure 6 but as an INTERACTIVE 3D plot](#Figure-6-but-as-an-INTERACTIVE-3D-plot)
	* [Figure 7](#Figure-7)
		* [Figure 7 but as an INTERACTIVE 3D plot](#Figure-7-but-as-an-INTERACTIVE-3D-plot)
* [Diffusion Approximation](#Diffusion-Approximation)
	* [Figure 8](#Figure-8)
	* [Figure 10](#Figure-10)
		* [Figure 10 but as an INTERACTIVE 3D plot](#Figure-10-but-as-an-INTERACTIVE-3D-plot)


Figure 3 is excluded because it is a diagram and does not involve data.

# Load dataset

In [ ]:
data = np.load("twostreams_revisited_data.npz")

(  # Inputs
    g_data,
    tau0_data,
    omega_data,
    mu0_data,
    Cloud_Leg_coeffs,
    omega_cloud_data,
    # Reflectance
    R_true_data,
    R_opt2_data,
    R_opt4_data,
    R_Zdun_data,
    R_Edd_data,
    R_CC1_data,
    R_true_cloud_data,
    R_HGopt2_cloud_data,
    R_HGopt4_cloud_data,
    R_Zdun_cloud_data,
    R_Edd_cloud_data,
    R_CC1_cloud_data,
    # Transmittance
    T_true_data,
    T_opt2_data,
    T_opt4_data,
    T_Zdun_data,
    T_Edd_data,
    T_CC1_data,
    T_true_cloud_data,
    T_HGopt2_cloud_data,
    T_HGopt4_cloud_data,
    T_Zdun_cloud_data,
    T_Edd_cloud_data,
    T_CC1_cloud_data,
    # Optimized gammas
    gamma1_opt_data,
    gamma2_opt_data,
    gamma1p_opt_data,
    gamma1m_opt_data,
    gamma2p_opt_data,
    gamma2m_opt_data,
    gamma1_opt_cloudbutHGomega_data,
    gamma2_opt_cloudbutHGomega_data,
    gamma1p_opt_cloudbutHGomega_data,
    gamma1m_opt_cloudbutHGomega_data,
    gamma2p_opt_cloudbutHGomega_data,
    gamma2m_opt_cloudbutHGomega_data,
    # Diffusion approximation
    a_s_plus,
    a_s_minus,
    gamma2m_tau1,
    gamma1p_tau1,
    gamma2m_tau10,
    gamma1p_tau10,
) = (values for values in data.values())

# Parameters and setup

Should realistic cloud phase functions and single-scattering albedos be used instead?

In [ ]:
use_cloud = False

**Choose problem parameters**

In [ ]:
g = np.random.choice(g_data)
tau0 = np.random.choice(tau0_data)
omega = np.random.choice(omega_data)
mu0 = np.random.choice(mu0_data)

if use_cloud:
    g_index = np.random.randint(6)
    g = Cloud_Leg_coeffs[g_index, 1]
    omega = omega_cloud_data[g_index]

print(g, tau0, omega, mu0)

## Figure 2

In [ ]:
# Plot problem paramaters (Fig. 2)
fig = plt.figure(figsize=(12, 6), dpi=200)
ax1 = plt.subplot(411)
plt.plot([0.62, 1], np.ones(2), color="lightgrey")
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.get_yaxis().set_ticks([])
plt.eventplot(g_data, orientation='horizontal')
plt.xticks(fontsize=15)
plt.ylabel("g", fontsize=20, rotation=0)
ax1.tick_params(labelleft=False)
ax2 = plt.subplot(412)
plt.plot([0.72, 1], np.ones(2), color="lightgrey")
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.get_yaxis().set_ticks([])
plt.eventplot(omega_data, orientation='horizontal')
plt.xticks(fontsize=15)
plt.ylabel(r"$\omega$", fontsize=20, rotation=0)
ax2.tick_params(labelleft=False)
ax3 = plt.subplot(413)
plt.plot([0, 11], np.ones(2), color="lightgrey")
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.get_yaxis().set_ticks([])
plt.eventplot(tau0_data, orientation='horizontal')
plt.xticks(fontsize=15)
plt.ylabel(r"$\tau_0$", fontsize=20, rotation=0)
ax3.tick_params(labelleft=False)
ax4 = plt.subplot(414)
plt.plot([0, 1], np.ones(2), color="lightgrey")
ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.spines['bottom'].set_visible(False)
ax4.spines['left'].set_visible(False)
ax4.get_yaxis().set_ticks([])
plt.eventplot(mu0_data, orientation='horizontal')
plt.xticks(fontsize=15)
plt.ylabel(r"$\mu_0$", fontsize=20, rotation=0)
plt.xlabel("Parameter value", fontsize=20)
ax4.tick_params(labelleft=False)

fig.tight_layout(h_pad=0)

**Choose computational parameters**

In [ ]:
if use_cloud:
    NQuad = 499 * 2 # Should choose prime number X 2
else:
    NQuad = 146 # Should choose prime number X 2
NLeg = NQuad

Ntau = 200  # Number of tau test points, mostly for plotting
tau_arr = np.linspace(0, tau0, Ntau)

In [ ]:
# Call high resolution solver
if use_cloud:
    Leg_coeffs_all = Cloud_Leg_coeffs[g_index, :] # Realistic cloud phase function Legendre coefficients
else:
    Leg_coeffs_all = g ** np.arange(250) # Henyey-Greenstein phase function Legendre coefficients
I0 = 1 / mu0

flux_up, flux_down, u0 = PythonicDISORT.pydisort(
    tau0,
    omega,
    NQuad,
    Leg_coeffs_all,
    mu0,
    I0,
    0,
    only_flux=True,
    autograd_compatible=True, # Change to `_autograd_compatible=True` for PythonicDISORT v0.9.1 and earlier
)[1:4]

N = NQuad // 2
mu_arr_pos, weights_mu = PythonicDISORT.subroutines.Gauss_Legendre_quad(N)

In [ ]:
# Compute backscattering ratio beta0
def S(Leg_coeffs):
    NLeg = len(Leg_coeffs)
    ell = np.arange(1, NLeg, 2, dtype="int64")
    t_ell = (-1) ** ((ell - 1) / 2) * (
        sc.special.factorial2(ell, exact=True)
        / (sc.special.factorial2(ell - 1, exact=True) * ell * (ell + 1))
    )
    coeffs = np.zeros(NLeg)
    coeffs[ell] = (2 * ell + 1) * t_ell * Leg_coeffs[ell]
    return Legendre(coeffs)


beta = lambda mu0: (1 / 2) * (
    1 + S(Leg_coeffs_all)(-mu0)
)  # Backscattering ratio function
beta0 = beta(mu0)
print("beta0 =", beta0)

**Formula (12) to compute fluxes and consequently reflectance and transmittance (and absorption)**

In [ ]:
# Formula (12) to compute fluxes
def gen_constcoeffs_diffusefluxes(
    g1p, g1m, g2p, g2m, g3, incident_flux, omega, tau0, mu0, g4=None
):
    if g4 == None:
        g4 = 1 - g3
    
    k = np.sqrt(g1p**2 + 2 * g1p * g1m + g1m**2 - 4 * g2p * g2m + 0j)
    lp = (g1p - g1m + k) / 2
    lm = (g1p - g1m - k) / 2
    a1 = 2 * g2m * ((g1p * g4 + g2p * g3) * mu0 + g4)
    a2 = 2 * g2p * ((g1m * g3 + g2m * g4) * mu0 - g3)
    a3 = g1p + g1m + k
    a4 = g1p + g1m - k
    a5 = (g1m * g3 + g2m * g4) * mu0 - g3
    a6 = (g1p * g4 + g2p * g3) * mu0 + g4
    a7 = (g1p * g1m - g2p * g2m) * mu0**2 + (g1m - g1p) * mu0 - 1
    #print(lp, lm)

    if np.real(lp) >= 0 and np.real(lm) >= 0:
        denom = a7 * (a3 * np.exp(-lm * tau0) - a4 * np.exp(-lp * tau0))
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        Fp = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * (tau - tau0)) - np.exp(lm * (tau - tau0)))
                - a5
                * (
                    a3
                    * (
                        np.exp(lp * (tau - tau0) - tau0 / mu0 - lm * tau0)
                        - np.exp(-tau / mu0 - lm * tau0)
                    )
                    - a4
                    * (
                        np.exp(lm * (tau - tau0) - tau0 / mu0 - lp * tau0)
                        - np.exp(-tau / mu0 - lp * tau0)
                    )
                )
            )
            * denom_reciprocal
        )
        Fm = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * (tau - tau0) - tau0 / mu0 - lp * tau0)
                    - np.exp(lp * (tau - tau0) - tau0 / mu0 - lm * tau0)
                )
                + a6
                * (
                    a3 * (np.exp(-tau / mu0 - lm * tau0) - np.exp(lm * (tau - tau0)))
                    - a4 * (np.exp(-tau / mu0 - lp * tau0) - np.exp(lp * (tau - tau0)))
                )
            )
            * denom_reciprocal
        )

    elif np.real(lp) >= 0 and np.real(lm) <= 0:
        denom = a7 * (a3 - a4 * np.exp(-k * tau0))  # lm - lp = -k
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        Fp = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * tau - k * tau0) - np.exp(lm * tau))
                - a5
                * (
                    a3 * (np.exp(lp * (tau - tau0) - tau0 / mu0) - np.exp(-tau / mu0))
                    - a4
                    * (
                        np.exp(lm * tau - tau0 / mu0 - lp * tau0)
                        - np.exp(-k * tau0 - tau / mu0)
                    )
                )
            )
            * denom_reciprocal
        )
        Fm = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * tau - tau0 / mu0 - lp * tau0)
                    - np.exp(lp * (tau - tau0) - tau0 / mu0)
                )
                + a6
                * (
                    a3 * (np.exp(-tau / mu0) - np.exp(lm * tau))
                    - a4 * (np.exp(-k * tau0 - tau / mu0) - np.exp(lp * tau - k * tau0))
                )
            )
            * denom_reciprocal
        )

    elif np.real(lp) <= 0 and np.real(lm) >= 0:
        denom = a7 * (a3 * np.exp(k * tau0) - a4)  # lp - lm = k
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        Fp = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * tau) - np.exp(k * tau0 + lm * tau))
                - a5
                * (
                    a3
                    * (
                        np.exp(lp * tau - tau0 / mu0 - lm * tau0)
                        - np.exp(k * tau0 - tau / mu0)
                    )
                    - a4 * (np.exp(lm * (tau - tau0) - tau0 / mu0) - np.exp(-tau / mu0))
                )
            )
            * denom_reciprocal
        )
        Fm = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * (tau - tau0) - tau0 / mu0)
                    - np.exp(lp * tau - tau0 / mu0 - lm * tau0)
                )
                + a6
                * (
                    a3 * (np.exp(k * tau0 - tau / mu0) - np.exp(k * tau0 + lm * tau))
                    - a4 * (np.exp(-tau / mu0) - np.exp(lp * tau))
                )
            )
            * denom_reciprocal
        )

    else:
        denom = a7 * (a3 * np.exp(lp * tau0) - a4 * np.exp(lm * tau0))
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        Fp = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * tau + lm * tau0) - np.exp(lp * tau0 + lm * tau))
                - a5
                * (
                    a3
                    * (
                        np.exp((lp * mu0 * tau - tau0) / mu0)
                        - np.exp((lp * mu0 * tau0 - tau) / mu0)
                    )
                    - a4
                    * (
                        np.exp((lm * mu0 * tau - tau0) / mu0)
                        - np.exp((lm * mu0 * tau0 - tau) / mu0)
                    )
                )
            )
            * denom_reciprocal
        )
        Fm = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp((lm * mu0 * tau - tau0) / mu0)
                    - np.exp((lp * mu0 * tau - tau0) / mu0)
                )
                + a6
                * (
                    a3
                    * (
                        np.exp((lp * mu0 * tau0 - tau) / mu0)
                        - np.exp(lp * tau0 + lm * tau)
                    )
                    - a4
                    * (
                        np.exp((lm * mu0 * tau0 - tau) / mu0)
                        - np.exp(lp * tau + lm * tau0)
                    )
                )
            )
            * denom_reciprocal
        )
        
    return Fp, Fm

In [ ]:
# Function to compute R, T (and A)
def RTA_constcoeffs_diffusefluxes(g1p, g1m, g2p, g2m, g3, omega, tau0, mu0, g4=None):
    if g4 == None:
        g4 = 1 - g3

    Fp, Fm = gen_constcoeffs_diffusefluxes(
        g1p, g1m, g2p, g2m, g3, 1, omega, tau0, mu0, g4
    )

    R = Fp(0)
    T = Fm(tau0)
    A = 1 - R - T
    return R, T, A

# Exact coupling coefficients

In [ ]:
# Implement exact coupling coefficients
S_cache = S(Leg_coeffs_all)(mu_arr_pos)

D1p = (
    lambda tau: (2 * pi)
    * (1 - omega / 2 * (1 + S_cache))
    * weights_mu
    @ atleast_2d_append(u0(tau))[:N, :]
)
D2m = (
    lambda tau: (2 * pi)
    * (omega / 2)
    * (1 - S_cache)
    * weights_mu
    @ atleast_2d_append(u0(tau))[N:, :]
)
D2p = (
    lambda tau: (2 * pi)
    * (omega / 2)
    * (1 - S_cache)
    * weights_mu
    @ atleast_2d_append(u0(tau))[:N, :]
)
D1m = (
    lambda tau: (2 * pi)
    * (1 - omega / 2 * (1 + S_cache))
    * weights_mu
    @ atleast_2d_append(u0(tau))[N:, :]
)


def gamma1p(tau):
    tau = np.atleast_1d(tau)
    Fp = flux_up(tau)
    return np.squeeze(
        np.divide(
            D1p(tau),
            Fp,
            out=np.zeros_like(tau, dtype=np.float64),
            where=Fp != 0,
            casting="unsafe",
        )
    )[()]


def gamma1m(tau):
    tau = np.atleast_1d(tau)
    Fm = flux_down(tau)[0]
    return np.squeeze(
        np.divide(
            D1m(tau),
            Fm,
            out=np.zeros_like(tau, dtype=np.float64),
            where=Fm != 0,
            casting="unsafe",
        )
    )[()]


def gamma2p(tau):
    tau = np.atleast_1d(tau)
    Fp = flux_up(tau)
    return np.squeeze(
        np.divide(
            D2p(tau),
            Fp,
            out=np.zeros_like(tau, dtype=np.float64),
            where=Fp != 0,
            casting="unsafe",
        )
    )[()]


def gamma2m(tau):
    tau = np.atleast_1d(tau)
    Fm = flux_down(tau)[0]
    return np.squeeze(
        np.divide(
            D2m(tau),
            Fm,
            out=np.zeros_like(tau, dtype=np.float64),
            where=Fm != 0,
            casting="unsafe",
        )
    )[()]


def gamma1(tau):
    tau = np.atleast_1d(tau)
    Fm = flux_down(tau)[0]
    Fp = flux_up(tau)
    denom = (Fp - Fm) * (Fp + Fm)
    return np.squeeze(
        np.divide(
            Fp * (D1p(tau) - D2m(tau)) + Fm * (D2p(tau) - D1m(tau)),
            denom,
            out=np.zeros_like(tau, dtype=np.float64),
            where=np.abs(denom) != 0,
            casting="unsafe",
        )
    )[()]


def gamma2(tau):
    tau = np.atleast_1d(tau)
    Fm = flux_down(tau)[0]
    Fp = flux_up(tau)
    denom = (Fp - Fm) * (Fp + Fm)
    return np.squeeze(
        np.divide(
            Fm * (D1p(tau) - D2m(tau)) + Fp * (D2p(tau) - D1m(tau)),
            denom,
            out=np.zeros_like(tau, dtype=np.float64),
            where=np.abs(denom) != 0,
            casting="unsafe",
        )
    )[()]

**Verify exact coupling coefficients**

In [ ]:
# Verify exact general two-stream equations
'''def RHS_exact(tau, F):
    plus = (
        gamma1p(tau) * F[0, :]
        - gamma2m(tau) * F[1, :]
        - omega * I0 * beta0 * np.exp(-tau / mu0)
    )
    minus = (
        gamma2p(tau) * F[0, :]
        - gamma1m(tau) * F[1, :]
        + omega * I0 * (1 - beta0) * np.exp(-tau / mu0)
    )
    return np.vstack((plus, minus))


sol_exact = sc.integrate.solve_bvp(
    RHS_exact,
    lambda ya, yb: np.array([ya[-1], yb[0]]),
    tau_arr,
    np.zeros((2, Ntau)),
    tol=1e-5,
    max_nodes=int(1e4),
    verbose=2,
)

print()
print(
    "Down flux max pointwise error =",
    np.max(np.abs(flux_up(tau_arr)[:-1] - sol_exact.sol(tau_arr)[0, :-1])),
)
print(
    "Up flux max pointwise error =",
    np.max(np.abs(flux_down(tau_arr)[0][:-1] - sol_exact.sol(tau_arr)[1, :-1])),
)
'''

In [ ]:
# Verify exact symmetric two-stream equations (decrease `tol` if this does not converge)
'''def RHS_exact(tau, F):
    plus = (
        gamma1(tau) * F[0, :]
        - gamma2(tau) * F[1, :]
        - omega * I0 * beta0 * np.exp(-tau / mu0)
    )
    minus = (
        gamma2(tau) * F[0, :]
        - gamma1(tau) * F[1, :]
        + omega * I0 * (1 - beta0) * np.exp(-tau / mu0)
    )
    return np.vstack((plus, minus))


sol_exact = sc.integrate.solve_bvp(
    RHS_exact,
    lambda ya, yb: np.array([ya[-1], yb[0]]),
    tau_arr,
    np.zeros((2, Ntau)),
    tol=1e-3,
    max_nodes=int(1e4),
    verbose=2,
)

print()
print(
    "Down flux max pointwise error =",
    np.max(np.abs(flux_up(tau_arr)[:-1] - sol_exact.sol(tau_arr)[0, :-1])),
)
print(
    "Up flux max pointwise error =",
    np.max(np.abs(flux_down(tau_arr)[0][:-1] - sol_exact.sol(tau_arr)[1, :-1])),
)
'''

**Plot exact coefficients**

In [ ]:
# Analytically continue and compute exact coupling coefficients
gamma1p_tau = gamma1p(tau_arr[:-1])
gamma1p_tau = np.append(
    gamma1p_tau,
    ((2 * pi) * (1 - omega / 2 * (1 + S_cache)) * weights_mu@ ag.jacobian(u0)(tau0)[:N])
    / ag.grad(flux_up)(tau0),
)
gamma1m_tau = gamma1m(tau_arr[1:])
gamma1m_tau = np.insert(
    gamma1m_tau,
    0,
    ((2 * pi) * (1 - omega / 2 * (1 + S_cache)) * weights_mu @ ag.jacobian(u0)(0.0)[N:])
    / ag.grad(lambda tau: flux_down(tau)[0])(0.0),
)
gamma2p_tau = gamma2p(tau_arr[:-1])
gamma2p_tau = np.append(
    gamma2p_tau,
    ((2 * pi) * (omega / 2) * (1 - S_cache) * weights_mu @ ag.jacobian(u0)(tau0)[:N])
    / ag.grad(flux_up)(tau0),
)
gamma2m_tau = gamma2m(tau_arr[1:])
gamma2m_tau = np.insert(
    gamma2m_tau,
    0,
    ((2 * pi) * (omega / 2) * (1 - S_cache) * weights_mu @ ag.jacobian(u0)(0.0)[N:])
    / ag.grad(lambda tau: flux_down(tau)[0])(0.0),
)

gamma1_tau = gamma1(tau_arr)
gamma2_tau = gamma2(tau_arr)

In [ ]:
# Compute net flux and discontinuity of symmetric coupling coefficients
F = lambda tau: flux_up(tau) - flux_down(tau)[0]
F_tau = F(tau_arr)
try:
    discontinuity = sc.optimize.root_scalar(
        F,
        bracket=(np.max(tau_arr[F_tau > 0]), np.min(tau_arr[F_tau < 0])),
    ).root
except:
    None

## Figure 1

In [ ]:
# Plot exact coupling coefficients (Fig. 1)
fig = plt.figure(figsize=(12, 12))
ax1 = plt.subplot(121)
plt.suptitle(
    "Exact coefficients at"
    + " $g =$"
    + str(np.round(g, 2))
    + r", $\omega =$"
    + str(np.round(omega, 2))
    + r", $\mu_0 =$"
    + str(np.round(mu0, 2))
    + r", $\tau_0 =$"
    + str(np.round(tau0, 2)),
    fontsize=23,
)
plt.axhline(tau0, color="lightgrey")
plt.axhline(0, color="lightgrey")
plt.plot(gamma2p_tau, tau_arr, '--', label="$\gamma_2^+$", color="darkblue")
plt.plot(gamma1p_tau, tau_arr, '--', label="$\gamma_1^+$", color="darkred")
plt.plot(gamma2m_tau, tau_arr, label="$\gamma_2^-$", color="slateblue")
plt.plot(gamma1m_tau, tau_arr, label="$\gamma_1^-$", color="salmon")
plt.yticks(fontsize=20)
plt.ylabel(r"Optical depth $\tau \in [0, \tau_0]$", fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel("General coefficient value", fontsize=20)
plt.gca().invert_yaxis()
plt.legend(fontsize=20)

ax2 = plt.subplot(122)
plt.axvline(0, color="lightgrey")
plt.axhline(tau0, color="lightgrey")
plt.axhline(0, color="lightgrey")
try:
    plt.axhline(
        discontinuity,
        linestyle='--',
        color="black",
        label="$\gamma_{1,2}$ asymptote",
    )
except:
    None
plt.plot(gamma1_tau[F_tau < 0], tau_arr[F_tau < 0], label="$\gamma_1$", color="red")
plt.plot(gamma2_tau[F_tau < 0], tau_arr[F_tau < 0], label="$\gamma_2$", color="blue")
plt.plot(gamma1_tau[F_tau > 0], tau_arr[F_tau > 0], color="red")
plt.plot(gamma2_tau[F_tau > 0], tau_arr[F_tau > 0], color="blue")
plt.xlim([-10, 10])
plt.gca().invert_yaxis()
plt.xticks(fontsize=20)
plt.xlabel("Symmetric coefficient value", fontsize=20)
ax2.tick_params(labelleft=False)
plt.legend(fontsize=20)
fig.tight_layout(w_pad=0)

# Baseline two-stream closures

In [ ]:
# Baseline two-stream closures
# Zdunkowski
def generate_Zdun_gammas(omega, g, mu0):
    g1 = (8 - omega * (5 + 3 * g)) / 4
    g2 = 3 * omega * (1 - g) / 4
    g3 = (2 - 3 * mu0 * g) / 4
    g4 = 1 - g3
    return g1, g2, g3, g4


# Eddington
def generate_Edd_gammas(omega, g, mu0):
    g1 = (7 - omega * (4 + 3 * g)) / 4
    g2 = -(1 - omega * (4 - 3 * g)) / 4
    g3 = (2 - 3 * mu0 * g) / 4
    g4 = 1 - g3
    return g1, g2, g3, g4


# Coakley-Chylek (I)
def generate_CC1_gammas(omega, g, mu0, beta):
    g1 = (1 - omega * (1 - beta(mu0))) / mu0
    g2 = omega * beta(mu0) / mu0
    g3 = beta(mu0)
    g4 = 1 - g3
    return g1, g2, g3, g4

In [ ]:
# Delta-Eddington scaling
f = Leg_coeffs_all[2]
g_scaled = (g - f) / (1 - f)
scale_tau = 1 - omega * f
omega_scaled = ((1 - f) / scale_tau) * omega
tau0_scaled = scale_tau * tau0

T_DSreclass = np.exp(-tau0_scaled / mu0) - np.exp (-tau0 / mu0)
Fm_DSreclass = (
    lambda tau: I0 * mu0 * (np.exp(-tau * scale_tau / mu0) - np.exp(-tau / mu0))
)

In [ ]:
# Compute coefficients for baseline two-stream methods
(
    gamma1_Zdun,
    gamma2_Zdun,
    gamma3_Zdun,
    gamma4_Zdun,
) = generate_Zdun_gammas(omega_scaled, g_scaled, mu0)

(
    gamma1_Edd,
    gamma2_Edd,
    gamma3_Edd,
    gamma4_Edd,
) = generate_Edd_gammas(omega_scaled, g_scaled, mu0)

(
    gamma1_CC1,
    gamma2_CC1,
    gamma3_CC1,
    gamma4_CC1,
) = generate_CC1_gammas(omega, g, mu0, beta)

In [ ]:
# Calculate R, T from baseline methods
R_true = flux_up(0)
T_true = flux_down(tau0)[0]

R_Zdun, T_Zdun, A_Zdun = RTA_constcoeffs_diffusefluxes(
    gamma1_Zdun,
    gamma1_Zdun,
    gamma2_Zdun,
    gamma2_Zdun,
    gamma3_Zdun,
    omega_scaled,
    tau0_scaled,
    mu0,
)
T_Zdun += T_DSreclass

R_Edd, T_Edd, A_Edd = RTA_constcoeffs_diffusefluxes(
    gamma1_Edd,
    gamma1_Edd,
    gamma2_Edd,
    gamma2_Edd,
    gamma3_Edd,
    omega_scaled,
    tau0_scaled,
    mu0,
)
T_Edd += T_DSreclass

R_CC1, T_CC1, A_CC1 = RTA_constcoeffs_diffusefluxes(
    gamma1_CC1,
    gamma1_CC1,
    gamma2_CC1,
    gamma2_CC1,
    gamma3_CC1,
    omega,
    tau0,
    mu0,
)

In [ ]:
# Print baseline R, T errors
print("REFLECTANCE")
print("True:", R_true)
print("Zdunkowski:", R_Zdun)
print(
    "( Error =",
    np.abs(R_Zdun - R_true),
    "|",
    "Error ratio = ",
    np.abs(R_Zdun - R_true) / R_true,
    ")",
)
print("Eddington:", R_Edd)
print(
    "( Error =",
    np.abs(R_Edd - R_true),
    "|",
    "Error ratio = ",
    np.abs(R_Edd - R_true) / R_true,
    ")",
)
print("Coakley-Chylek (I):", R_CC1)
print(
    "( Error =",
    np.abs(R_CC1 - R_true),
    "|",
    "Error ratio = ",
    np.abs(R_CC1 - R_true) / R_true,
    ")",
)
print()
print("TRANSMITTANCE")
print("True:", T_true)
print("Zdunkowski:", T_Zdun)
print(
    "( Error =",
    np.abs(T_Zdun - T_true),
    "|",
    "Error ratio = ",
    np.abs(T_Zdun - T_true) / T_true,
    ")",
)
print("Eddington:", T_Edd)
print(
    "( Error =",
    np.abs(T_Edd - T_true),
    "|",
    "Error ratio = ",
    np.abs(T_Edd - T_true) / T_true,
    ")",
)
print("Coakley-Chylek (I):", T_CC1)
print(
    "( Error =",
    np.abs(T_CC1 - T_true),
    "|",
    "Error ratio = ",
    np.abs(T_CC1 - T_true) / T_true,
    ")",
)

## Figure 4

In [ ]:
# Baseline two-stream closure errors with respect to Henyey-Greenstein (Fig. 4); data must be manually keyed
baseline_methods = (
    "Valid for Coakley-Chylek (I) \n (Thin atmospheres)",
    """ "Difficult" \n (Medium-thick atmospheres) """,
    "Valid for Eddington \n (Thick atmospheres)",
)
R = {
    "Zdun R": (60, 27, 15),
    "Edd R": (60, 27, 11),
    "CC(I) R": (7, 39, 39),
}

T = {
    "Zdun T": (-23, -20, -8),
    "Edd T": (-23, -20, -9),
    "CC(I) T": (-6, -67, -43),
}

x = np.arange(len(baseline_methods))  # the label locations
width = 0.2  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout="constrained", dpi=300)

for attribute, measurement in R.items():
    offset = width * multiplier
    rects = ax.bar(
        x + offset,
        measurement,
        width,
        label=attribute,
        edgecolor="black",
        color=colors[multiplier + 2],
    )
    multiplier += 1

multiplier = 0
bottom_index = 0
for attribute, measurement in T.items():
    offset = width * multiplier
    rects = ax.bar(
        x + offset,
        measurement,
        width,
        label=attribute,
        edgecolor="black",
        color=["lightgreen", "lightcoral", "plum"][multiplier],
    )
    multiplier += 1

ax.set_ylabel("Worst-case percentage error", fontsize=12)
ax.set_xticks(x + width, baseline_methods)
plt.axhline(0, color="black")
plt.legend()

## Figure 9

In [ ]:
# Two-stream closure errors with respect to a cloud (Fig. 9); data must be manually keyed
baseline_methods = (
    "Valid for Coakley-Chylek (I) \n (Thin clouds)",
    """ "Difficult" \n (Medium-thick clouds) """,
    "Valid for Eddington \n (Thick clouds)",
)
R = {
    "Edd R": (41, 19, 8),
    "CC(I) R": (0.4, 33, 32),
    "Sym R": (30, 5, 2),
}

T = {
    "Edd T": (-11, -19, -5),
    "CC(I) T": (-0.7, -57, -23),
    "Sym T": (-8, -3, -0.5),
    
}

x = np.arange(len(baseline_methods))  # the label locations
width = 0.2  # the width of the bars
multiplier = 0
bar_colors = [colors[3], colors[4], colors[0]]

fig, ax = plt.subplots(layout="constrained", dpi=300)

for attribute, measurement in R.items():
    offset = width * multiplier
    rects = ax.bar(
        x + offset,
        measurement,
        width,
        label=attribute,
        edgecolor="black",
        color=bar_colors[multiplier],
    )
    multiplier += 1

multiplier = 0
bottom_index = 0
for attribute, measurement in T.items():
    offset = width * multiplier
    rects = ax.bar(
        x + offset,
        measurement,
        width,
        label=attribute,
        edgecolor="black",
        color=["lightcoral", "plum", "lightblue"][multiplier],
    )
    multiplier += 1

ax.set_ylabel("Worst-case percentage error", fontsize=12)
ax.set_xticks(x + width, baseline_methods)
plt.axhline(0, color="black")
plt.legend()

In [ ]:
# R, T error over all problem parameters

clip_min = 0

print("Worst case error / error ratios with respect to Henyey-Greenstein")
print("-------------------------------")
print("Coakley-Chylek")
print(
    "Transmittivity:",
    np.max(np.abs((T_CC1_data - T_true_data))),
    "/",
    np.max(np.abs((T_CC1_data - T_true_data) / T_true_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_CC1_data - R_true_data))),
    "/",
    np.max(np.abs((R_CC1_data - R_true_data) / R_true_data.clip(min=clip_min))),
)

print()
print("Eddington")
print(
    "Transmittivity:",
    np.max(np.abs((T_Edd_data - T_true_data))),
    "/",
    np.max(np.abs((T_Edd_data - T_true_data) / T_true_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_Edd_data - R_true_data))),
    "/",
    np.max(np.abs((R_Edd_data - R_true_data) / R_true_data.clip(min=clip_min))),
)

print()
print("Zdunkowski")
print(
    "Transmittivity:",
    np.max(np.abs((T_Zdun_data - T_true_data))),
    "/",
    np.max(np.abs((T_Zdun_data - T_true_data) / T_true_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_Zdun_data - R_true_data))),
    "/",
    np.max(np.abs((R_Zdun_data - R_true_data) / R_true_data.clip(min=clip_min))),
)

print()
print("Symmetric HG-optimized")
print(
    "Transmittivity:",
    np.max(np.abs((T_opt2_data - T_true_data))),
    "/",
    np.max(np.abs((T_opt2_data - T_true_data) / T_true_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_opt2_data - R_true_data))),
    "/",
    np.max(np.abs((R_opt2_data - R_true_data) / R_true_data.clip(min=clip_min))),
)

print()
print("General HG-optimized")
print(
    "Transmittivity:",
    np.max(np.abs((T_opt4_data - T_true_data))),
    "/",
    np.max(np.abs((T_opt4_data - T_true_data) / T_true_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_opt4_data - R_true_data))),
    "/",
    np.max(np.abs((R_opt4_data - R_true_data) / R_true_data.clip(min=clip_min))),
)
print()
print()

print("Worst case error / error ratios with respect to clouds")
print("-------------------------------")
print("Coakley-Chylek")
print(
    "Transmittivity:",
    np.max(np.abs((T_CC1_cloud_data - T_true_cloud_data))),
    "/",
    np.max(np.abs((T_CC1_cloud_data - T_true_cloud_data) / T_true_cloud_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_CC1_cloud_data - R_true_cloud_data))),
    "/",
    np.max(np.abs((R_CC1_cloud_data - R_true_cloud_data) / R_true_cloud_data.clip(min=clip_min))),
)

print()
print("Eddington")
print(
    "Transmittivity:",
    np.max(np.abs((T_Edd_cloud_data - T_true_cloud_data))),
    "/",
    np.max(np.abs((T_Edd_cloud_data - T_true_cloud_data) / T_true_cloud_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_Edd_cloud_data - R_true_cloud_data))),
    "/",
    np.max(np.abs((R_Edd_cloud_data - R_true_cloud_data) / R_true_cloud_data.clip(min=clip_min))),
)

print()
print("Zdunkowski")
print(
    "Transmittivity:",
    np.max(np.abs((T_Zdun_cloud_data - T_true_cloud_data))),
    "/",
    np.max(np.abs((T_Zdun_cloud_data - T_true_cloud_data) / T_true_cloud_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_Zdun_cloud_data - R_true_cloud_data))),
    "/",
    np.max(np.abs((R_Zdun_cloud_data - R_true_cloud_data) / R_true_cloud_data.clip(min=clip_min))),
)

print()
print("Symmetric HG-optimized")
print(
    "Transmittivity:",
    np.max(np.abs((T_HGopt2_cloud_data - T_true_cloud_data))),
    "/",
    np.max(np.abs((T_HGopt2_cloud_data - T_true_cloud_data) / T_true_cloud_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_HGopt2_cloud_data - R_true_cloud_data))),
    "/",
    np.max(np.abs((R_HGopt2_cloud_data - R_true_cloud_data) / R_true_cloud_data.clip(min=clip_min))),
)

print()
print("General HG-optimized")
print(
    "Transmittivity:",
    np.max(np.abs((T_HGopt4_cloud_data - T_true_cloud_data))),
    "/",
    np.max(np.abs((T_HGopt4_cloud_data - T_true_cloud_data) / T_true_cloud_data.clip(min=clip_min))),
)
print(
    "Reflectivity:",
    np.max(np.abs((R_HGopt4_cloud_data - R_true_cloud_data))),
    "/",
    np.max(np.abs((R_HGopt4_cloud_data - R_true_cloud_data) / R_true_cloud_data.clip(min=clip_min))),
)

# Numerical Optimization (for one set of problem parameters)

In [ ]:
# Vertically-integrated flux formulas, i.e. equation (12) integrated from tau = 0 to tau = tau0
def gen_constcoeffs_indefint_diffusefluxes(
    g1p, g1m, g2p, g2m, g3, incident_flux, omega, tau0, mu0, g4=None
):
    if g4 == None:
        g4 = 1 - g3

    k = np.sqrt(g1p**2 + 2 * g1p * g1m + g1m**2 - 4 * g2p * g2m + 0j)
    lp = (g1p - g1m + k) / 2
    lm = (g1p - g1m - k) / 2
    a1 = 2 * g2m * ((g1p * g4 + g2p * g3) * mu0 + g4)
    a2 = 2 * g2p * ((g1m * g3 + g2m * g4) * mu0 - g3)
    a3 = g1p + g1m + k
    a4 = g1p + g1m - k
    a5 = (g1m * g3 + g2m * g4) * mu0 - g3
    a6 = (g1p * g4 + g2p * g3) * mu0 + g4
    a7 = (g1p * g1m - g2p * g2m) * mu0**2 + (g1m - g1p) * mu0 - 1
    
    if np.abs(lp) < 1e-15:
        lp_reciprocal = 0
    else:
        lp_reciprocal = 1 / lp
    if np.abs(lm) < 1e-15:
        lm_reciprocal = 0
    else:
        lm_reciprocal = 1 / lm

    if np.real(lp) >= 0 and np.real(lm) >= 0:
        denom = a7 * (a3 * np.exp(-lm * tau0) - a4 * np.exp(-lp * tau0))
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        IntFpC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * (tau - tau0)) * lp_reciprocal - np.exp(lm * (tau - tau0)) * lm_reciprocal)
                - a5
                * (
                    a3
                    * (
                        np.exp(lp * (tau - tau0) - tau0 / mu0 - lm * tau0) * lp_reciprocal
                        + mu0 * np.exp(-tau / mu0 - lm * tau0)
                    )
                    - a4
                    * (
                        np.exp(lm * (tau - tau0) - tau0 / mu0 - lp * tau0) * lm_reciprocal
                        + mu0 * np.exp(-tau / mu0 - lp * tau0)
                    )
                )
            )
            * denom_reciprocal
        )
        IntFmC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * (tau - tau0) - tau0 / mu0 - lp * tau0) * lm_reciprocal
                    - np.exp(lp * (tau - tau0) - tau0 / mu0 - lm * tau0) * lp_reciprocal
                )
                + a6
                * (
                    a3
                    * (
                        -mu0 * np.exp(-tau / mu0 - lm * tau0)
                        - np.exp(lm * (tau - tau0)) * lm_reciprocal
                    )
                    - a4
                    * (
                        -mu0 * np.exp(-tau / mu0 - lp * tau0)
                        - np.exp(lp * (tau - tau0)) * lp_reciprocal
                    )
                )
            )
            * denom_reciprocal
        )

    elif np.real(lp) >= 0 and np.real(lm) <= 0:
        denom = a7 * (a3 - a4 * np.exp(-k * tau0))  # lm - lp = -k
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        IntFpC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * tau - k * tau0) * lp_reciprocal - np.exp(lm * tau) * lm_reciprocal)
                - a5
                * (
                    a3
                    * (
                        np.exp(lp * (tau - tau0) - tau0 / mu0) * lp_reciprocal
                        + mu0 * np.exp(-tau / mu0)
                    )
                    - a4
                    * (
                        np.exp(lm * tau - tau0 / mu0 - lp * tau0) * lm_reciprocal
                        + mu0 * np.exp(-k * tau0 - tau / mu0)
                    )
                )
            )
            * denom_reciprocal
        )
        IntFmC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * tau - tau0 / mu0 - lp * tau0) * lm_reciprocal
                    - np.exp(lp * (tau - tau0) - tau0 / mu0) * lp_reciprocal
                )
                + a6
                * (
                    a3 * (-mu0 * np.exp(-tau / mu0) - np.exp(lm * tau) * lm_reciprocal)
                    - a4
                    * (
                        -mu0 * np.exp(-k * tau0 - tau / mu0)
                        - np.exp(lp * tau - k * tau0) * lp_reciprocal
                    )
                )
            )
            * denom_reciprocal
        )

    elif np.real(lp) <= 0 and np.real(lm) >= 0:
        denom = a7 * (a3 * np.exp(k * tau0) - a4)  # lp - lm = k
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        IntFpC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1 * (np.exp(lp * tau) * lp_reciprocal - np.exp(k * tau0 + lm * tau) * lm_reciprocal)
                - a5
                * (
                    a3
                    * (
                        np.exp(lp * tau - tau0 / mu0 - lm * tau0) * lp_reciprocal
                        + mu0 * np.exp(k * tau0 - tau / mu0)
                    )
                    - a4
                    * (
                        np.exp(lm * (tau - tau0) - tau0 / mu0) * lm_reciprocal
                        + mu0 * np.exp(-tau / mu0)
                    )
                )
            )
            * denom_reciprocal
        )
        IntFmC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * (tau - tau0) - tau0 / mu0) * lm_reciprocal
                    - np.exp(lp * tau - tau0 / mu0 - lm * tau0) * lp_reciprocal
                )
                + a6
                * (
                    a3
                    * (
                        -mu0 * np.exp(k * tau0 - tau / mu0)
                        - np.exp(k * tau0 + lm * tau) * lm_reciprocal
                    )
                    - a4 * (-mu0 * np.exp(-tau / mu0) - np.exp(lp * tau) * lp_reciprocal)
                )
            )
            * denom_reciprocal
        )

    else:
        denom = a7 * (a3 * np.exp(lp * tau0) - a4 * np.exp(lm * tau0))
        if np.abs(denom) < 1e-15:
            denom_reciprocal = 0
        else:
            denom_reciprocal = 1 / denom
        IntFpC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a1
                * (
                    np.exp(lp * tau + lm * tau0) * lp_reciprocal
                    - np.exp(lp * tau0 + lm * tau) * lm_reciprocal
                )
                - a5
                * (
                    a3
                    * (
                        np.exp(lp * tau - tau0 / mu0) * lp_reciprocal
                        + mu0 * np.exp(lp * tau0 - tau / mu0)
                    )
                    - a4
                    * (
                        np.exp(lm * tau - tau0 / mu0) * lm_reciprocal
                        + mu0 * np.exp(lm * tau0 - tau / mu0)
                    )
                )
            )
            * denom_reciprocal
        )
        IntFmC = lambda tau: np.real(
            omega
            * incident_flux
            * (
                a2
                * (
                    np.exp(lm * tau - tau0 / mu0) * lm_reciprocal
                    - np.exp(lp * tau - tau0 / mu0) * lp_reciprocal
                )
                + a6
                * (
                    a3
                    * (
                        -mu0 * np.exp(lp * tau0 - tau / mu0)
                        - np.exp(lp * tau0 + lm * tau) * lm_reciprocal
                    )
                    - a4
                    * (
                        -mu0 * np.exp(lm * tau0 - tau / mu0)
                        - np.exp(lp * tau + lm * tau0) * lp_reciprocal
                    )
                )
            )
            * denom_reciprocal
        )
        
    return IntFpC, IntFmC

In [ ]:
# Vertically-integrated D terms (of equation (6))
intu0 = (u0(tau0, is_antiderivative_wrt_tau=True) - u0(0, is_antiderivative_wrt_tau=True))

dFtrue1p = (
    -(2 * pi)
    * (1 - omega / 2 * (1 + S_cache))
    * weights_mu
    @ atleast_2d_append(intu0)[:N, :]
)[0]
dFtrue2m = (
    -(2 * pi)
    * (omega / 2)
    * (1 - S_cache)
    * weights_mu
    @ atleast_2d_append(intu0)[N:, :]
)[0]
dFtrue2p = (
    (2 * pi)
    * (omega / 2)
    * (1 - S_cache)
    * weights_mu
    @ atleast_2d_append(intu0)[:N, :]
)[0]
dFtrue1m = (
    (2 * pi)
    * (1 - omega / 2 * (1 + S_cache))
    * weights_mu
    @ atleast_2d_append(intu0)[N:, :]
)[0]

**Optimize general coupling coefficients**

In [ ]:
# Objective function and acceptance test
tau_check = np.linspace(0, tau0, 51)[1:-1]


def to_min4(gamma_arr):
    intFpC, intFmC = gen_constcoeffs_indefint_diffusefluxes(
        gamma_arr[0],
        gamma_arr[1],
        gamma_arr[2],
        gamma_arr[3],
        beta0,
        I0 * mu0,
        omega,
        tau0,
        mu0,
    )
    intFp = intFpC(tau0) - intFpC(0)
    intFm = intFmC(tau0) - intFmC(0)
    dF1p = gamma_arr[0] * -intFp
    dF1m = gamma_arr[1] * intFm
    dF2p = gamma_arr[2] * intFp
    dF2m = gamma_arr[3] * -intFm
    return np.sum(
        np.array(
            [
                (dF1p - dFtrue1p) / dFtrue1p,
                (dF1m - dFtrue1m) / dFtrue1m,
                (dF2p - dFtrue2p) / dFtrue2p,
                (dF2m - dFtrue2m) / dFtrue2m,
            ]
        )
        ** 2
    )


jac4 = ag.grad(to_min4)


def accept_test4(f_new, x_new, f_old, x_old):
    Fp, Fm = gen_constcoeffs_diffusefluxes(
        x_new[0],
        x_new[1],
        x_new[2],
        x_new[3],
        beta0,
        I0 * mu0,
        omega,
        tau0,
        mu0,
    )
    return (
        np.all(x_new > 0)
        and np.all(Fp(tau_check) > 0)
        and np.all(Fm(tau_check) > 0)
    )

In [ ]:
# Optimization
IC = np.random.random(4)
print("Initial condition =", IC)

opt4 = sc.optimize.basinhopping(
    to_min4,
    IC,
    accept_test=accept_test4,
    minimizer_kwargs={"jac": jac4},
    T=0,
    stepsize=0.3,
    niter=1000,
    niter_success=50,
    interval=20,
    target_accept_rate=0.3,
    disp=True,
)

In [ ]:
# Result
gamma1p_opt, gamma1m_opt, gamma2p_opt, gamma2m_opt = opt4["x"]
opt4

**Optimized symmetric coupling coefficients**

In [ ]:
# Objective function
tau_check = np.linspace(0, tau0, 51)[1:-1]


def to_min2(gamma_arr):
    intFpC, intFmC = gen_constcoeffs_indefint_diffusefluxes(
        gamma_arr[0],
        gamma_arr[0],
        gamma_arr[1],
        gamma_arr[1],
        beta0,
        I0 * mu0,
        omega,
        tau0,
        mu0,
    )
    intFp = intFpC(tau0) - intFpC(0)
    intFm = intFmC(tau0) - intFmC(0)
    dF1p = gamma_arr[0] * -intFp
    dF1m = gamma_arr[0] * intFm
    dF2p = gamma_arr[1] * intFp
    dF2m = gamma_arr[1] * -intFm
    return np.sum(
        np.array(
            [
                ((dF1p - dF2m) - (dFtrue1p - dFtrue2m)) / (dFtrue1p - dFtrue2m),
                ((dF2p - dF1m) - (dFtrue2p - dFtrue1m)) / (dFtrue2p - dFtrue1m),
            ]
        )
        ** 2
    )


jac2 = ag.grad(to_min2)


def accept_test2(f_new, x_new, f_old, x_old):
    Fp, Fm = gen_constcoeffs_diffusefluxes(
        x_new[0],
        x_new[0],
        x_new[1],
        x_new[1],
        beta0,
        I0 * mu0,
        omega,
        tau0,
        mu0,
    )
    return (
        np.all(x_new > 0)
        and np.all(Fp(tau_check) > 0)
        and np.all(Fm(tau_check) > 0)
    )

In [ ]:
# Optimization
IC = np.random.random(2)
print("Initial condition =", IC)

opt2 = sc.optimize.basinhopping(
    to_min2,
    IC,
    accept_test=accept_test2,
    minimizer_kwargs={"jac": jac2},
    T=0,
    stepsize=0.3,
    niter=1000,
    niter_success=50,
    interval=20,
    target_accept_rate=0.3,
    disp=True,
)

In [ ]:
# Result
gamma1_opt, gamma2_opt = opt2["x"]
opt2

In [ ]:
# Calculate R, T from optimized methods
R_opt2, T_opt2, A_opt2 = RTA_constcoeffs_diffusefluxes(
    gamma1_opt,
    gamma1_opt,
    gamma2_opt,
    gamma2_opt,
    beta0,
    omega,
    tau0,
    mu0,
)

R_opt4, T_opt4, A_opt4 = RTA_constcoeffs_diffusefluxes(
    gamma1p_opt,
    gamma1m_opt,
    gamma2p_opt,
    gamma2m_opt,
    beta0,
    omega,
    tau0,
    mu0,
)

In [ ]:
# Print R, T errors
print("REFLECTANCE")
print("True:", R_true)
print("Local optimization")
print("Optimized (2 coefficients):", R_opt2)
print(
    "( Error =",
    np.abs(R_opt2 - R_true),
    "|",
    "Error ratio = ",
    np.abs(R_opt2 - R_true) / R_true,
    ")",
)
print("Optimized (4 coefficients):", R_opt4)
print(
    "( Error =",
    np.abs(R_opt4 - R_true),
    "|",
    "Error ratio = ",
    np.abs(R_opt4 - R_true) / R_true,
    ")",
)
print()
print("TRANSMITTANCE")
print("True:", T_true)
print("Optimized (2 coefficients):", T_opt2)
print(
    "( Error =",
    np.abs(T_opt2 - T_true),
    "|",
    "Error ratio = ",
    np.abs(T_opt2 - T_true) / T_true,
    ")",
)
print("Optimized (4 coefficients):", T_opt4)
print(
    "( Error =",
    np.abs(T_opt4 - T_true),
    "|",
    "Error ratio = ",
    np.abs(T_opt4 - T_true) / T_true,
    ")",
)

In [ ]:
# Generate data to plot Fig. 4
Fp2, Fm2 = gen_constcoeffs_diffusefluxes(
    gamma1_opt,
    gamma1_opt,
    gamma2_opt,
    gamma2_opt,
    beta0,
    I0 * mu0,
    omega,
    tau0,
    mu0,
)
Fp2_abs_err = np.abs(Fp2(tau_arr) - flux_up(tau_arr))
Fm2_abs_err = np.abs(Fm2(tau_arr) - flux_down(tau_arr)[0])
Fp2_rel_err = np.abs(Fp2(tau_arr) - flux_up(tau_arr)) / flux_up(tau_arr)
Fm2_rel_err = np.abs(Fm2(tau_arr) - flux_down(tau_arr)[0]) / flux_down(tau_arr)[0]

Fp4, Fm4 = gen_constcoeffs_diffusefluxes(
    gamma1p_opt,
    gamma1m_opt,
    gamma2p_opt,
    gamma2m_opt,
    beta0,
    I0 * mu0,
    omega,
    tau0,
    mu0,
)
Fp4_abs_err = np.abs(Fp4(tau_arr) - flux_up(tau_arr))
Fm4_abs_err = np.abs(Fm4(tau_arr) - flux_down(tau_arr)[0])
Fp4_rel_err = np.abs(Fp4(tau_arr) - flux_up(tau_arr)) / flux_up(tau_arr)
Fm4_rel_err = np.abs(Fm4(tau_arr) - flux_down(tau_arr)[0]) / flux_down(tau_arr)[0]

Fp_Zdun, Fm_Zdun = gen_constcoeffs_diffusefluxes(
    gamma1_Zdun,
    gamma1_Zdun,
    gamma2_Zdun,
    gamma2_Zdun,
    gamma3_Zdun,
    I0 * mu0,
    omega_scaled,
    tau0_scaled,
    mu0,
)
Fp_Zdun_abs_err = np.abs(Fp_Zdun(tau_arr * scale_tau) - flux_up(tau_arr))
Fm_Zdun_abs_err = np.abs(
    Fm_Zdun(tau_arr * scale_tau) + Fm_DSreclass(tau_arr) - flux_down(tau_arr)[0]
)
Fp_Zdun_rel_err = np.abs(Fp_Zdun(tau_arr * scale_tau) - flux_up(tau_arr)) / flux_up(
    tau_arr
)
Fm_Zdun_rel_err = (
    np.abs(Fm_Zdun(tau_arr * scale_tau) + Fm_DSreclass(tau_arr) - flux_down(tau_arr)[0])
    / flux_down(tau_arr)[0]
)

Fp_Edd, Fm_Edd = gen_constcoeffs_diffusefluxes(
    gamma1_Edd,
    gamma1_Edd,
    gamma2_Edd,
    gamma2_Edd,
    gamma3_Edd,
    I0 * mu0,
    omega_scaled,
    tau0_scaled,
    mu0,
)
Fp_Edd_abs_err = np.abs(Fp_Edd(tau_arr * scale_tau) - flux_up(tau_arr))
Fm_Edd_abs_err = np.abs(
    Fm_Edd(tau_arr * scale_tau) + Fm_DSreclass(tau_arr) - flux_down(tau_arr)[0]
)
Fp_Edd_rel_err = np.abs(Fp_Edd(tau_arr * scale_tau) - flux_up(tau_arr)) / flux_up(
    tau_arr
)
Fm_Edd_rel_err = (
    np.abs(Fm_Edd(tau_arr * scale_tau) + Fm_DSreclass(tau_arr) - flux_down(tau_arr)[0])
    / flux_down(tau_arr)[0]
)

Fp_CC1, Fm_CC1 = gen_constcoeffs_diffusefluxes(
    gamma1_CC1,
    gamma1_CC1,
    gamma2_CC1,
    gamma2_CC1,
    gamma3_CC1,
    I0 * mu0,
    omega,
    tau0,
    mu0,
)
Fp_CC1_abs_err = np.abs(Fp_CC1(tau_arr) - flux_up(tau_arr))
Fm_CC1_abs_err = np.abs(Fm_CC1(tau_arr) - flux_down(tau_arr)[0])
Fp_CC1_rel_err = np.abs(Fp_CC1(tau_arr) - flux_up(tau_arr)) / flux_up(tau_arr)
Fm_CC1_rel_err = np.abs(Fm_CC1(tau_arr) - flux_down(tau_arr)[0]) / flux_down(tau_arr)[0]

## Figure 5

In [ ]:
# Plots of flux profiles and errors (Fig. 4)
fig = plt.figure(figsize=(12, 12))
plt.suptitle(
    "Flux profiles at"
    + " $g =$"
    + str(np.round(g, 2))
    + r", $\omega =$"
    + str(np.round(omega, 2))
    + r", $\mu_0 =$"
    + str(np.round(mu0, 2))
    + r", $\tau_0 =$"
    + str(np.round(tau0, 2)),
    fontsize=23,
)

ax1 = plt.subplot(121)
plt.axhline(tau0, color="lightgrey")
plt.axhline(0, color="lightgrey")
plt.plot(flux_up(tau_arr), tau_arr, color="blue", label="True upward flux")
plt.plot(flux_down(tau_arr)[0], tau_arr, '--', color="blue", label="True downward flux")
plt.yticks(fontsize=20)
plt.ylabel(r"Optical depth $\tau \in [0, \tau_0]$", fontsize=20)
plt.xlabel(r"Flux profiles", fontsize=20)
plt.locator_params(axis='x', nbins=6)
plt.xticks(fontsize=20)
plt.legend(fontsize=20)
plt.gca().invert_yaxis()
plt.legend(fontsize=20)

ax2 = plt.subplot(122)
ax2.tick_params(labelleft=False)
plt.axhline(tau0, color="lightgrey")
plt.axhline(0, color="lightgrey")
plt.plot(Fp2_abs_err, tau_arr, color=colors[0], label="Symmetric (up)")
plt.plot(Fm2_abs_err, tau_arr, "--", color=colors[0], label="Symmetric (down)")
plt.plot(Fp4_abs_err, tau_arr, color=colors[1], label="General (up)")
plt.plot(Fm4_abs_err, tau_arr, "--", color=colors[1], label="General (down)")
plt.plot(Fp_Zdun_abs_err, tau_arr, color=colors[2], label="Zdunkowski (up)")
plt.plot(Fm_Zdun_abs_err, tau_arr, "--", color=colors[2], label="Zdunkowski (down)")
plt.locator_params(axis='x', nbins=6)
plt.yticks(fontsize=20)
plt.xlabel(r"Pointwise absolute errors", fontsize=20)
plt.xticks(fontsize=20)
plt.gca().invert_yaxis()
plt.legend(fontsize=20)

#plt.legend(fontsize=20)
fig.tight_layout(w_pad=0)

# Contour and 3D plots of dataset

In [ ]:
eff_radius = np.linspace(4, 20, 6)

# Create meshgrids
T, M = np.meshgrid(np.log10(tau0_data), mu0_data)
G, O = np.meshgrid(g_data, omega_data)
R, O_cloud = np.meshgrid(eff_radius, omega_data)

## Figure 6

In [ ]:
# Variation against single-scattering parameters: g, omega (Fig. 6)
fig = plt.figure(figsize=(12, 10), dpi=200)
ax1 = plt.subplot((221))
ax1.tick_params(labelbottom=False)
CS = plt.contourf(
    G,
    O,
    gamma2m_opt_data[:, -1, :, 0].T,
    levels=15,
)
plt.title(r"$\gamma_2^-$; $\tau = 0.01$, $\mu_0 = 1$", fontsize=20)
plt.ylabel(r"$\omega$", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(np.linspace(0.75, 1, 6), fontsize=15)
cbar = fig.colorbar(CS)
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_yticks(np.linspace(0, 0.6, 7))

ax2 = plt.subplot((222))
ax2.tick_params(labelleft=False)
ax2.tick_params(labelbottom=False)
CS2 = plt.contourf(
    G,
    O,
    gamma2p_opt_data[:, 0, :, -1].T,
    levels=15,
)
plt.title(r"$\gamma_2^+$; $\tau = 10$, $\mu_0 = 0.01$", fontsize=20)
cbar2 = fig.colorbar(CS2)
cbar2.ax.tick_params(labelsize=15)
cbar2.ax.set_yticks(np.linspace(0.2, 0.5, 4))

#####

ax3 = plt.subplot((223))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
CS3 = plt.contourf(
    G, 
    O, 
    gamma2_opt_data[:, -1, :, 0].T,
    levels=15,
    extend="min"
)
plt.title(r"$\gamma_2$; $\tau = 0.01$, $\mu_0 = 1$", fontsize=20)
plt.xlabel(r"$g$", fontsize=20)
plt.ylabel(r"$\omega$", fontsize=20)
CS3.cmap.set_under("white")
cbar3 = fig.colorbar(CS3)
cbar3.ax.tick_params(labelsize=15)
cbar3.ax.set_yticks(np.linspace(0, 0.2, 3))

ax4 = plt.subplot((224))
ax4.tick_params(labelleft=False)
CS4 = plt.contourf(
    G, 
    O, 
    gamma2_opt_data[:, 0, :, -1].T,
    levels=15,
)
plt.title(r"$\gamma_2$; $\tau = 10$, $\mu_0 = 0.01$", fontsize=20)
plt.xlabel(r"$g$", fontsize=20)
plt.xticks(fontsize=15)
CS4.cmap.set_under("white")
cbar4 = fig.colorbar(CS4)
cbar4.ax.tick_params(labelsize=15)
cbar4.ax.set_yticks(np.linspace(0.2, 0.4, 3))

fig.tight_layout(w_pad=0)

### Figure 6 but as an INTERACTIVE 3D plot

In [ ]:
# Choose which optimized coupling coefficient to plot out of
'''
General:   gamma1p_opt_data, gamma1m_opt_data, gamma2p_opt_data, gamma2m_opt_data,
Symmetric: gamma1_opt_data, gamma2_opt_data
'''

plot_go = gamma1p_opt_data

*WARNING: this plot is computationally intensive and may not load. Clearing all outputs before running this code block should help.*

In [ ]:
# INTERACTIVE 3D plot of the variation against single-scattering parameters: g, omega (Fig. 6)
'''Nt, Nm = len(tau0_data), len(mu0_data)

layout = go.Layout(
    scene=dict(
        zaxis=dict(title="Coefficient value"),
        xaxis=dict(title="Single-scattering albedo"),
        yaxis=dict(title="Asymmetry parameter"),
        aspectratio=dict(x=1.5, y=1.5, z=1),
    ),
    font=dict(size=15)
)
fig = go.Figure(layout=layout)

# Add traces, one for each slider step
for step in range(Nt):
    fig.add_trace(
        go.Surface(
            visible=False,
            x=omega_data,
            y=g_data,
            z=plot_go[:, step, :, 0],
            hovertemplate=("omega: %{x:.3f}<br>g: %{y:.3f}<br><b>gamma: %{z:.3f}<b>"),
        )
    )

# Make 0th trace visible
fig.data[0].visible = True

# Create and add slider for tau0
steps_tau0 = []
for i in range(Nt):
    step = dict(
        method="update",
        args=[
            {"visible": [False] * Nt},
        ],
        label=str(np.around(tau0_data[i], 3)),
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps_tau0.append(step)

# Create and add slider for mu0
steps_mu0 = []
for i in range(Nm):
    plot_go_it = np.moveaxis(plot_go[:, :, :, i], 1, 0)
    step = dict(
        method="update",
        args=[
            {"z": plot_go_it},
        ],
        label=str(np.around(mu0_data[i], 3)),
    )
    steps_mu0.append(step)

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "tau0: "},
        steps=steps_tau0,
    ),
    dict(
        active=0,
        currentvalue={"prefix": "mu0: "},
        steps=steps_mu0,
        pad={"t": 100},
    ),
]

fig.update_layout(
    sliders=sliders,
    width=1000,
    height=800,
)'''

## Figure 7

In [ ]:
# Variation against macro parameters: tau0, mu0 (Fig. 7)
fig = plt.figure(figsize=(12, 15), dpi=200)
ax1 = plt.subplot((321))
ax1.tick_params(labelbottom=False)
CS = plt.contourf(
    T,
    M,
    gamma1_opt_data[0, :, -3, :].T,
    np.concatenate(
        (
            np.linspace(0, 2, 11)[:-1],
            np.arange(2, np.max(gamma1_opt_data[0, :, -3, :]), 0.2),
        )
    ),
    norm=mpl.colors.TwoSlopeNorm(
        vmin=0, vcenter=2, vmax=np.max(gamma1_opt_data[0, :, -3, :])
    ),
    cmap=mpl.colormaps["pink"],
)
plt.title(r"$g = 0.65$", fontsize=25)
plt.ylabel(r"$\mu_0$", fontsize=20)
plt.yticks(fontsize=15)
cbar = fig.colorbar(CS)
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_yticks(np.linspace(1, 3, 3))
cbar.ax.axhline(2, c="r")

ax2 = plt.subplot((322))
ax2.tick_params(labelleft=False, labelbottom=False)
CS2 = plt.contourf(
    T,
    M,
    gamma1_opt_data[-1, :, -3, :].T,
    np.concatenate(
        (
            np.linspace(0, 2, 11)[:-1],
            np.arange(2, np.max(gamma1_opt_data[-1, :, -3, :]), 0.2),
        )
    ),
    norm=mpl.colors.TwoSlopeNorm(
        vmin=0, vcenter=2, vmax=np.max(gamma1_opt_data[-1, :, -3, :])
    ),
    cmap=mpl.colormaps["pink"],
)
plt.title(r"$g = 0.95$", fontsize=25)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
CS2.cmap.set_under("black")
cbar2 = fig.colorbar(CS2)
cbar2.ax.tick_params(labelsize=15)
cbar2.ax.set_yticks(np.linspace(1, 11, 6))
cbar2.ax.axhline(2, c="r")

#####

ax3 = plt.subplot((323))
ax3.tick_params(labelbottom=False)
CS3 = plt.contourf(
    T,
    M,
    gamma1p_opt_data[0, :, -3, :].T,
    np.concatenate(
        (
            np.linspace(0, 2, 11)[:-1],
            np.arange(2, np.max(gamma1p_opt_data[0, :, -3, :]), 0.2),
        )
    ),
    norm=mpl.colors.TwoSlopeNorm(
        vmin=0, vcenter=2, vmax=np.max(gamma1p_opt_data[0, :, -3, :])
    ),
    cmap=mpl.colormaps["pink"],
)
plt.ylabel(r"$\mu_0$", fontsize=20)
plt.yticks(fontsize=15)
cbar3 = fig.colorbar(CS3)
cbar3.ax.tick_params(labelsize=15)
cbar3.ax.set_yticks(np.linspace(1, 4, 4))
cbar3.ax.axhline(2, c="r")

ax4 = plt.subplot((324))
ax4.tick_params(labelleft=False, labelbottom=False)
CS4 = plt.contourf(
    T,
    M,
    gamma1p_opt_data[-1, :, -3, :].T,
    np.concatenate(
        (
            np.linspace(0, 2, 11)[:-1],
            np.arange(2, np.max(gamma1p_opt_data[-1, :, -3, :]), 0.2),
        )
    ),
    norm=mpl.colors.TwoSlopeNorm(
        vmin=0, vcenter=2, vmax=np.max(gamma1p_opt_data[-1, :, -3, :])
    ),
    cmap=mpl.colormaps["pink"],
)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
CS4.cmap.set_under("black")
cbar4 = fig.colorbar(CS4)
cbar4.ax.tick_params(labelsize=15)
cbar4.ax.set_yticks(np.linspace(2, 18, 9))
cbar4.ax.axhline(2, c="r")

#####

ax5 = plt.subplot((325))
CS5 = plt.contourf(
    T,
    M,
    gamma1m_opt_data[0, :, -3, :].T,
    np.concatenate(
        (
            np.linspace(0, 2, 11)[:-1],
            np.arange(2, np.max(gamma1m_opt_data[0, :, -3, :]), 0.2),
        )
    ),
    norm=mpl.colors.TwoSlopeNorm(
        vmin=0, vcenter=2, vmax=np.max(gamma1m_opt_data[0, :, -3, :])
    ),
    cmap=mpl.colormaps["pink"],
)
plt.xlabel(r"$\tau_0$", fontsize=20)
plt.ylabel(r"$\mu_0$", fontsize=20)
plt.xticks(
    np.arange(-2, 2, 1),
    labels=[r"$0.01$", r"$0.1$", r"$1$", r"$10$"],
    fontsize=15,
)
plt.yticks(fontsize=15)
cbar5 = fig.colorbar(CS5)
cbar5.ax.tick_params(labelsize=15)
cbar5.ax.set_yticks(np.linspace(1, 4, 4))
cbar5.ax.axhline(2, c="r")

ax6 = plt.subplot((326))
ax6.tick_params(labelleft=False)
CS6 = plt.contourf(
    T,
    M,
    gamma1m_opt_data[-1, :, -3, :].T,
    np.concatenate(
        (
            np.linspace(0, 2, 11)[:-1],
            np.arange(2, np.max(gamma1m_opt_data[-1, :, -3, :]), 0.2),
        )
    ),
    norm=mpl.colors.TwoSlopeNorm(
        vmin=0, vcenter=2, vmax=np.max(gamma1m_opt_data[-1, :, -3, :])
    ),
    cmap=mpl.colormaps["pink"],
)
plt.xlabel(r"$\tau_0$", fontsize=20)
plt.xticks(
    np.arange(-2, 2, 1),
    labels=[r"$0.01$", r"$0.1$", r"$1$", r"$10$"],
    fontsize=15,
)
plt.yticks(fontsize=15)
CS6.cmap.set_under("black")
cbar6 = fig.colorbar(CS6)
cbar6.ax.tick_params(labelsize=15)
cbar6.ax.set_yticks(np.linspace(2, 14, 7))
cbar6.ax.axhline(2, c="r")

fig.tight_layout(w_pad=0)

### Figure 7 but as an INTERACTIVE 3D plot

In [ ]:
# Choose which optimized coupling coefficient to plot out of
'''
General:   gamma1p_opt_data, gamma1m_opt_data, gamma2p_opt_data, gamma2m_opt_data,
Symmetric: gamma1_opt_data, gamma2_opt_data
'''

plot_tm = gamma1p_opt_data

*WARNING: this plot is computationally intensive and may not load. Clearing all outputs before running this code block should help.*

In [ ]:
# INTERACTIVE 3D plot of the variation against macro parameters: tau0, mu0 (Fig. 7)
'''Ng, No = len(g_data), len(omega_data)

layout = go.Layout(
    scene=dict(
        zaxis=dict(title="Coefficient value"),
        xaxis=dict(title="Cosine of solar zenith angle"),
        yaxis=dict(title="Total optical depth"),
        aspectratio=dict(x=1.5, y=1.5, z=1),
    ),
    font=dict(size=15)
)
fig = go.Figure(layout=layout)

# Add traces, one for each slider step
for step in range(Ng):
    fig.add_trace(
        go.Surface(
            visible=False,
            x=mu0_data,
            y=tau0_data,
            z=plot_tm[step, :, 0, :],
            hovertemplate=("mu0: %{x:.3f}<br>tau0: %{y:.3f}<br><b>gamma: %{z:.3f}<b>"),
        )
    )

# Make 0th trace visible
fig.data[0].visible = True

# Create and add slider for g
steps_g = []
for i in range(Ng):
    step = dict(
        method="update",
        args=[
            {"visible": [False] * Ng},
        ],
        label=str(np.around(g_data[i], 3)),
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps_g.append(step)

# Create and add slider for omega
steps_omega = []
for i in range(No):
    plot_tm_io = plot_tm[:, :, i, :]
    step = dict(
        method="update",
        args=[
            {"z": plot_tm_io},
        ],
        label=str(np.around(omega_data[i], 6)),
    )
    steps_omega.append(step)

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "g: "},
        steps=steps_g,
    ),
    dict(
        active=0,
        currentvalue={"prefix": "omega: "},
        steps=steps_omega,
        pad={"t": 100},
    ),
]

fig.update_layout(
    sliders=sliders,
    width=1000,
    height=800,
)'''

# Diffusion Approximation

Plotting the diffusion approximation requires data that is not normally provided by PythonicDISORT, or its FORTRAN antecedent DISORT, but can be extracted from them. Here, we provide the necessary data to plot Fig. 8.

In [ ]:
# Setup for Fig. 8
omega = 0.7488113568490418
mu_arr_pos, weights_mu = PythonicDISORT.subroutines.Gauss_Legendre_quad(53)
S_cache_fig8 = S(0.65 ** np.arange(250))(mu_arr_pos)
tau_arr1 = np.linspace(0, 1, 200)
tau_arr10 = np.linspace(0, 10, 200)

In [ ]:
# Compute diffusion approximations
gamma1p_DA = ((1 - omega / 2 * (1 + S_cache_fig8)) * weights_mu @ a_s_plus) / (
    mu_arr_pos * weights_mu @ a_s_plus
)
gamma1m_DA = ((1 - omega / 2 * (1 + S_cache_fig8)) * weights_mu @ a_s_minus) / (
    mu_arr_pos * weights_mu @ a_s_minus
)
gamma2p_DA = ((omega / 2) * (1 - S_cache_fig8) * weights_mu @ a_s_plus) / (
    mu_arr_pos * weights_mu @ a_s_plus
)
gamma2m_DA = ((omega / 2) * (1 - S_cache_fig8) * weights_mu @ a_s_minus) / (
    mu_arr_pos * weights_mu @ a_s_minus
)

## Figure 8

In [ ]:
# Plot exact coupling coefficients with corresponding diffusion approximations (Fig. 8)
fig = plt.figure(figsize=(12, 12), dpi=200)
ax1 = plt.subplot((121))
plt.title(r"$\tau_0 = 1$", fontsize=25)
plt.gca().invert_yaxis()
plt.axhline(1, color="lightgrey")
plt.axhline(0, color="lightgrey")
plt.plot(gamma2m_tau1, tau_arr1, label=r"$\gamma_2^-$", color="royalblue")
plt.plot(gamma1p_tau1, tau_arr1, label=r"$\gamma_1^+$", color="red")
plt.plot(np.repeat(gamma2m_DA, len(tau_arr1)), tau_arr1, "-.", color="royalblue", label=r"$\gamma_2^-$ DA")
plt.plot(np.repeat(gamma1p_DA, len(tau_arr1)), tau_arr1, "-.", color="red", label=r"$\gamma_1^+$ DA")
plt.ylabel(r"Optical depth $\tau \in [0, \tau_0]$", fontsize=20)
plt.xlabel("Coefficient value", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)

ax2 = plt.subplot((122))
plt.title(r"$\tau_0 = 10$", fontsize=25)
plt.gca().invert_yaxis()
plt.axhline(10, color="lightgrey")
plt.axhline(0, color="lightgrey")
plt.plot(gamma2m_tau10, tau_arr10, label=r"$\gamma_2^-$", color="royalblue")
plt.plot(gamma1p_tau10, tau_arr10, label=r"$\gamma_1^+$", color="red")
plt.plot(np.repeat(gamma2m_DA, len(tau_arr10)), tau_arr10, "-.", color="royalblue", label=r"$\gamma_2^-$ DA")
plt.plot(np.repeat(gamma1p_DA, len(tau_arr10)), tau_arr10, "-.", color="red", label=r"$\gamma_1^+$ DA")
plt.xlabel("Coefficient value", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

fig.tight_layout(h_pad=0)

## Figure 10

In [ ]:
# Variation against single-scattering parameters: eff_radius, omega (Fig. 10)
fig = plt.figure(figsize=(12, 10), dpi=200)
ax1 = plt.subplot((221))
ax1.tick_params(labelbottom=False)
CS = plt.contourf(
    R,
    O_cloud,
    gamma2m_opt_cloudbutHGomega_data[:, -1, :, 8].T,
    levels=15,
)
plt.title(r"$\gamma_2^-$; $\tau = 0.01$, $\mu_0 = 0.45$", fontsize=20)
plt.ylabel(r"$\omega$", fontsize=20)
plt.xticks(eff_radius, fontsize=15)
plt.yticks(np.linspace(0.75, 1, 6), fontsize=15)
cbar = fig.colorbar(CS)
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_yticks(np.linspace(0.2, 0.1, 6))

ax2 = plt.subplot((222))
ax2.tick_params(labelleft=False)
ax2.tick_params(labelbottom=False)
CS2 = plt.contourf(
    R,
    O_cloud,
    gamma2p_opt_cloudbutHGomega_data[:, 0, :, -1].T,
    levels=15,
)
plt.title(r"$\gamma_2^+$; $\tau = 10$, $\mu_0 = 0.01$", fontsize=20)
plt.xticks(eff_radius, fontsize=15)
plt.yticks(np.linspace(0.75, 1, 6), fontsize=15)
cbar2 = fig.colorbar(CS2)
cbar2.ax.tick_params(labelsize=15)
cbar2.ax.set_yticks(np.linspace(0.16, 0.34, 7))

#####

ax3 = plt.subplot((223))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
CS3 = plt.contourf(
    R, 
    O_cloud, 
    gamma2_opt_cloudbutHGomega_data[:, -1, :, 17].T,
    levels=15,
    vmin=0,
    extend="min"
)
plt.title(r"$\gamma_2$; $\tau = 0.01$, $\mu_0 = 0.45$", fontsize=20)
plt.xticks(eff_radius, fontsize=15)
#ax4.set_xticklabels(["0.834", "0.852", "0.860", "0.864", "0.867", "0.869"])
plt.yticks(np.linspace(0.75, 1, 6), fontsize=15)
plt.xlabel(r"$r_\text{eff}$", fontsize=20)
plt.ylabel(r"$\omega$", fontsize=20)
CS3.cmap.set_under("white")
cbar3 = fig.colorbar(CS3)
cbar3.ax.tick_params(labelsize=15)
cbar3.ax.set_yticks(np.linspace(0.3, 0, 7))

ax4 = plt.subplot((224))
ax4.tick_params(labelleft=False)
CS4 = plt.contourf(
    R, 
    O_cloud, 
    gamma2_opt_cloudbutHGomega_data[:, 0, :, -1].T,
    levels=15,
)
plt.title(r"$\gamma_2$; $\tau = 10$, $\mu_0 = 0.01$", fontsize=20)
plt.xticks(eff_radius, fontsize=15)
plt.yticks(np.linspace(0.75, 1, 6), fontsize=15)
plt.xlabel(r"$r_\text{eff}$", fontsize=20)
plt.xticks(fontsize=15)
#ax4.set_xticklabels(["0.834", "0.852", "0.860", "0.864", "0.867", "0.869"])
CS4.cmap.set_under("white")
cbar4 = fig.colorbar(CS4)
cbar4.ax.tick_params(labelsize=15)
cbar4.ax.set_yticks(np.linspace(0.33, 0.21, 7))

fig.tight_layout(w_pad=0)

### Figure 10 but as an INTERACTIVE 3D plot

In [ ]:
# Choose which optimized coupling coefficient to plot out of
'''
General:   gamma1p_opt_cloudbutHGomega_data, gamma1m_opt_cloudbutHGomega_data, 
           gamma2p_opt_cloudbutHGomega_data, gamma2m_opt_cloudbutHGomega_data,
Symmetric: gamma1_opt_cloudbutHGomega_data, gamma2_opt_cloudbutHGomega_data
'''

plot_go_cloud = gamma2_opt_cloudbutHGomega_data

In [ ]:
# INTERACTIVE 3D plot of the variation against single-scattering parameters: g, omega (Fig. 10)
'''Nt, Nm = len(tau0_data), len(mu0_data)

layout = go.Layout(
    scene=dict(
        zaxis=dict(title="Coefficient value"),
        xaxis=dict(title="Single-scattering albedo"),
        yaxis=dict(title="Effective radius"),
        aspectratio=dict(x=1.5, y=1.5, z=1),
    ),
    font=dict(size=15)
)
fig = go.Figure(layout=layout)

# Add traces, one for each slider step
for step in range(Nt):
    fig.add_trace(
        go.Surface(
            visible=False,
            x=omega_data,
            y=eff_radius,
            z=plot_go_cloud[:, step, :, 0],
            hovertemplate=("omega: %{x:.3f}<br>r: %{y:.3f}<br><b>gamma: %{z:.3f}<b>"),
        )
    )

# Make 0th trace visible
fig.data[0].visible = True

# Create and add slider for tau0
steps_tau0 = []
for i in range(Nt):
    step = dict(
        method="update",
        args=[
            {"visible": [False] * Nt},
        ],
        label=str(np.around(tau0_data[i], 3)),
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps_tau0.append(step)

# Create and add slider for mu0
steps_mu0 = []
for i in range(Nm):
    plot_go_cloud_it = np.moveaxis(plot_go_cloud[:, :, :, i], 1, 0)
    step = dict(
        method="update",
        args=[
            {"z": plot_go_cloud_it},
        ],
        label=str(np.around(mu0_data[i], 3)),
    )
    steps_mu0.append(step)

sliders = [
    dict(
        active=0,
        currentvalue={"prefix": "tau0: "},
        steps=steps_tau0,
    ),
    dict(
        active=0,
        currentvalue={"prefix": "mu0: "},
        steps=steps_mu0,
        pad={"t": 100},
    ),
]

fig.update_layout(
    sliders=sliders,
    width=1000,
    height=800,
)'''